In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/update_stresult15-12-15.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st12-9-12/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [12, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 12], [12], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '9'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 12]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive100=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st12-9-12/tmp/model.ckpt')
                        togive4=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,12)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive4.append(output_any_image)
                            
                        togive100=togive4
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 0.976473153
Validation Loss: 1.4652545
Epoch: 0002 cost = 0.963836176
Validation Loss: 1.101263
Epoch: 0003 cost = 0.947522998
Validation Loss: 0.9765257
Epoch: 0004 cost = 0.927926472
Validation Loss: 0.9454007
Epoch: 0005 cost = 0.914936866
Validation Loss: 0.92948216
Epoch: 0006 cost = 0.905187760
Validation Loss: 0.9198495
Epoch: 0007 cost = 0.894946958
Validation Loss: 0.90986854
Epoch: 0008 cost = 0.884671007
Validation Loss: 0.8898213
Epoch: 0009 cost = 0.874791767
Validation Loss: 0.86199677
Epoch: 0010 cost = 0.864581985
Validation Loss: 0.82016385
Epoch: 0011 cost = 0.853540301
Validation Loss: 0.7907073
Epoch: 0012 cost = 0.841737245
Validation Loss: 0.7627163
Epoch: 0013 cost = 0.829981983
Validation Loss: 0.7474916
Epoch: 0014 cost = 0.818226550
Validation Loss: 0.73376745
Epoch: 0015 cost = 0.804797820
Validation Loss: 0.7160919
Epoch: 0016 cost = 0.788584513
Validation Loss: 0.69030046
Epoch: 0017 cost = 0.769694243
Validation Loss: 0.6558922


Epoch: 0141 cost = 0.099185266
Validation Loss: 0.15416309
Epoch: 0142 cost = 0.097124886
Validation Loss: 0.10781759
Epoch: 0143 cost = 0.095122021
Validation Loss: 0.1380645
Epoch: 0144 cost = 0.093176809
Validation Loss: 0.09469597
Epoch: 0145 cost = 0.091289025
Validation Loss: 0.13518299
Epoch: 0146 cost = 0.089458510
Validation Loss: 0.09436733
Epoch: 0147 cost = 0.087684995
Validation Loss: 0.13446857
Epoch: 0148 cost = 0.085968181
Validation Loss: 0.08728367
Epoch: 0149 cost = 0.084307807
Validation Loss: 0.12811768
Epoch: 0150 cost = 0.082703263
Validation Loss: 0.0825279
Epoch: 0151 cost = 0.081154324
Validation Loss: 0.12925373
Epoch: 0152 cost = 0.079659867
Validation Loss: 0.088815205
Epoch: 0153 cost = 0.078219698
Validation Loss: 0.0821591
Epoch: 0154 cost = 0.076832912
Validation Loss: 0.1335668
Epoch: 0155 cost = 0.075498466
Validation Loss: 0.0847767
Epoch: 0156 cost = 0.074215602
Validation Loss: 0.076698676
Epoch: 0157 cost = 0.072983167
Validation Loss: 0.12598135


Validation Loss: 0.038183104
Epoch: 0281 cost = 0.042691733
Validation Loss: 0.03701243
Epoch: 0282 cost = 0.042639564
Validation Loss: 0.034049425
Epoch: 0283 cost = 0.042587994
Validation Loss: 0.034000233
Epoch: 0284 cost = 0.042536850
Validation Loss: 0.033945803
Epoch: 0285 cost = 0.042486034
Validation Loss: 0.04028631
Epoch: 0286 cost = 0.042436052
Validation Loss: 0.04108187
Epoch: 0287 cost = 0.042386223
Validation Loss: 0.035254803
Epoch: 0288 cost = 0.042336938
Validation Loss: 0.034450028
Epoch: 0289 cost = 0.042288349
Validation Loss: 0.033485312
Epoch: 0290 cost = 0.042240190
Validation Loss: 0.032171883
Epoch: 0291 cost = 0.042192377
Validation Loss: 0.035069205
Epoch: 0292 cost = 0.042145044
Validation Loss: 0.043501355
Epoch: 0293 cost = 0.042098165
Validation Loss: 0.043454207
Epoch: 0294 cost = 0.042051863
Validation Loss: 0.047325335
Epoch: 0295 cost = 0.042006107
Validation Loss: 0.044474408
Epoch: 0296 cost = 0.041960647
Validation Loss: 0.039347257
Epoch: 0297 co

Epoch: 0424 cost = 0.038790603
Validation Loss: 0.02969259
Epoch: 0425 cost = 0.038776305
Validation Loss: 0.02543421
Epoch: 0426 cost = 0.038761899
Validation Loss: 0.0266067
Epoch: 0427 cost = 0.038747694
Validation Loss: 0.027160201
Epoch: 0428 cost = 0.038733645
Validation Loss: 0.023978287
Epoch: 0429 cost = 0.038719418
Validation Loss: 0.12737049
Epoch: 0430 cost = 0.038705403
Validation Loss: 0.049559552
Epoch: 0431 cost = 0.038691309
Validation Loss: 0.027769135
Epoch: 0432 cost = 0.038677487
Validation Loss: 0.029629266
Epoch: 0433 cost = 0.038663423
Validation Loss: 0.03133503
Epoch: 0434 cost = 0.038649592
Validation Loss: 0.03255034
Epoch: 0435 cost = 0.038635750
Validation Loss: 0.034591995
Epoch: 0436 cost = 0.038621850
Validation Loss: 0.035921965
Epoch: 0437 cost = 0.038608094
Validation Loss: 0.03600012
Epoch: 0438 cost = 0.038594353
Validation Loss: 0.029509127
Epoch: 0439 cost = 0.038580741
Validation Loss: 0.02797784
Epoch: 0440 cost = 0.038567036
Validation Loss: 0

Validation Loss: 0.030264007
Epoch: 0573 cost = 0.036789821
Validation Loss: 0.0317023
Epoch: 0574 cost = 0.036775398
Validation Loss: 0.03348582
Epoch: 0575 cost = 0.036760963
Validation Loss: 0.028706796
Epoch: 0576 cost = 0.036746447
Validation Loss: 0.024377428
Epoch: 0577 cost = 0.036731788
Validation Loss: 0.029555665
Epoch: 0578 cost = 0.036717254
Validation Loss: 0.028262295
Epoch: 0579 cost = 0.036702485
Validation Loss: 0.030843286
Epoch: 0580 cost = 0.036688066
Validation Loss: 0.033221584
Epoch: 0581 cost = 0.036673229
Validation Loss: 0.038601905
Epoch: 0582 cost = 0.036658629
Validation Loss: 0.03511993
Epoch: 0583 cost = 0.036643910
Validation Loss: 0.034251075
Epoch: 0584 cost = 0.036629299
Validation Loss: 0.044742174
Epoch: 0585 cost = 0.036614344
Validation Loss: 0.06201481
Epoch: 0586 cost = 0.036599640
Validation Loss: 0.06911957
Epoch: 0587 cost = 0.036584776
Validation Loss: 0.05977364
Epoch: 0588 cost = 0.036569843
Validation Loss: 0.047069214
Epoch: 0589 cost =

...

Validation Loss: 0.090633854
Epoch: 4385 cost = 0.027311495
Validation Loss: 0.04278483
Epoch: 4386 cost = 0.027301107
Validation Loss: 0.028852722
Epoch: 4387 cost = 0.027292721
Validation Loss: 0.026308438
Epoch: 4388 cost = 0.027284521
Validation Loss: 0.031445466
Epoch: 4389 cost = 0.027277715
Validation Loss: 0.035052046
Epoch: 4390 cost = 0.027271468
Validation Loss: 0.03850485
Epoch: 4391 cost = 0.027265146
Validation Loss: 0.04398505
Epoch: 4392 cost = 0.027259719
Validation Loss: 0.0665024
Epoch: 4393 cost = 0.027254398
Validation Loss: 0.05233256
Epoch: 4394 cost = 0.027248968
Validation Loss: 0.043292537
Epoch: 4395 cost = 0.027244493
Validation Loss: 0.03552528
Epoch: 4396 cost = 0.027241301
Validation Loss: 0.044329803
Epoch: 4397 cost = 0.027241288
Validation Loss: 0.040222224
Epoch: 4398 cost = 0.027249544
Validation Loss: 0.040367633
Epoch: 4399 cost = 0.027275721
Validation Loss: 0.037860267
Epoch: 4400 cost = 0.027335539
Validation Loss: 0.03604139
Epoch: 4401 cost = 

Validation Loss: 0.06290314
Epoch: 4523 cost = 0.027487470
Validation Loss: 0.06436684
Epoch: 4524 cost = 0.027395517
Validation Loss: 0.04582122
Epoch: 4525 cost = 0.027338878
Validation Loss: 0.029797703
Epoch: 4526 cost = 0.027309550
Validation Loss: 0.030973062
Epoch: 4527 cost = 0.027299163
Validation Loss: 0.058378145
Epoch: 4528 cost = 0.027303614
Validation Loss: 0.072302215
Epoch: 4529 cost = 0.027312405
Validation Loss: 0.08609262
Epoch: 4530 cost = 0.027315264
Validation Loss: 0.11710852
Epoch: 4531 cost = 0.027310992
Validation Loss: 0.101454414
Epoch: 4532 cost = 0.027301060
Validation Loss: 0.049196947
Epoch: 4533 cost = 0.027288739
Validation Loss: 0.043614764
Epoch: 4534 cost = 0.027275615
Validation Loss: 0.040258195
Epoch: 4535 cost = 0.027263000
Validation Loss: 0.033953574
Epoch: 4536 cost = 0.027251817
Validation Loss: 0.050455693
Epoch: 4537 cost = 0.027241733
Validation Loss: 0.13695998
Epoch: 4538 cost = 0.027233870
Validation Loss: 0.16178119
Epoch: 4539 cost =

Validation Loss: 0.05357916
Epoch: 4677 cost = 0.027256141
Validation Loss: 0.058441248
Epoch: 4678 cost = 0.027261905
Validation Loss: 0.0489646
Epoch: 4679 cost = 0.027261182
Validation Loss: 0.06530178
Epoch: 4680 cost = 0.027253590
Validation Loss: 0.095542625
Epoch: 4681 cost = 0.027242437
Validation Loss: 0.07902655
Epoch: 4682 cost = 0.027230276
Validation Loss: 0.10142557
Epoch: 4683 cost = 0.027217230
Validation Loss: 0.09185113
Epoch: 4684 cost = 0.027205645
Validation Loss: 0.06653168
Epoch: 4685 cost = 0.027195008
Validation Loss: 0.07551957
Epoch: 4686 cost = 0.027186140
Validation Loss: 0.04160768
Epoch: 4687 cost = 0.027178353
Validation Loss: 0.059247825
Epoch: 4688 cost = 0.027171160
Validation Loss: 0.07309191
Epoch: 4689 cost = 0.027164565
Validation Loss: 0.075370334
Epoch: 4690 cost = 0.027158469
Validation Loss: 0.076448224
Epoch: 4691 cost = 0.027152498
Validation Loss: 0.04995869
Epoch: 4692 cost = 0.027147410
Validation Loss: 0.052782107
Epoch: 4693 cost = 0.02

Validation Loss: 0.03806703
Epoch: 4830 cost = 0.027169043
Validation Loss: 0.043119904
Epoch: 4831 cost = 0.027157406
Validation Loss: 0.04273629
Epoch: 4832 cost = 0.027147044
Validation Loss: 0.045888525
Epoch: 4833 cost = 0.027137995
Validation Loss: 0.046993393
Epoch: 4834 cost = 0.027130045
Validation Loss: 0.048872028
Epoch: 4835 cost = 0.027122993
Validation Loss: 0.040489532
Epoch: 4836 cost = 0.027116220
Validation Loss: 0.027823465
Epoch: 4837 cost = 0.027109860
Validation Loss: 0.044923995
Epoch: 4838 cost = 0.027104007
Validation Loss: 0.033933036
Epoch: 4839 cost = 0.027099112
Validation Loss: 0.038132228
Epoch: 4840 cost = 0.027095647
Validation Loss: 0.043789193
Epoch: 4841 cost = 0.027095893
Validation Loss: 0.053813476
Epoch: 4842 cost = 0.027106222
Validation Loss: 0.054007955
Epoch: 4843 cost = 0.027137548
Validation Loss: 0.055048693
Epoch: 4844 cost = 0.027206034
Validation Loss: 0.0639469
Epoch: 4845 cost = 0.027304969
Validation Loss: 0.042889915
Epoch: 4846 cos

Epoch: 4970 cost = 0.027159951
Validation Loss: 0.048712417
Epoch: 4971 cost = 0.027166304
Validation Loss: 0.046569843
Epoch: 4972 cost = 0.027165081
Validation Loss: 0.041156385
Epoch: 4973 cost = 0.027158458
Validation Loss: 0.031077065
Epoch: 4974 cost = 0.027147767
Validation Loss: 0.052311473
Epoch: 4975 cost = 0.027135753
Validation Loss: 0.052530203
Epoch: 4976 cost = 0.027123792
Validation Loss: 0.06257586
Epoch: 4977 cost = 0.027111803
Validation Loss: 0.054714307
Epoch: 4978 cost = 0.027101559
Validation Loss: 0.04754257
Epoch: 4979 cost = 0.027092700
Validation Loss: 0.066460386
Epoch: 4980 cost = 0.027084629
Validation Loss: 0.06231214
Epoch: 4981 cost = 0.027077284
Validation Loss: 0.049973562
Epoch: 4982 cost = 0.027070095
Validation Loss: 0.03457638
Epoch: 4983 cost = 0.027064063
Validation Loss: 0.027672868
Epoch: 4984 cost = 0.027057996
Validation Loss: 0.041332684
Epoch: 4985 cost = 0.027053464
Validation Loss: 0.0640652
Epoch: 4986 cost = 0.027051035
Validation Loss

In [3]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult12-9-12.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive100)):
        wr.writerow(list(togive100[i][0]))

In [4]:
togive100

[array([[0.56900537, 0.5649609 , 0.5562152 , 0.46612626, 0.46785414,
         0.5187281 , 0.6255483 , 0.25576627, 0.6143002 ]], dtype=float32),
 array([[0.5616818 , 0.5514237 , 0.5525823 , 0.48475742, 0.47418287,
         0.50330484, 0.61213344, 0.27358383, 0.59887016]], dtype=float32),
 array([[0.5289277 , 0.48876527, 0.50979   , 0.5312016 , 0.50743645,
         0.45889294, 0.56159437, 0.36655068, 0.54105   ]], dtype=float32),
 array([[0.4450718 , 0.42074865, 0.42172593, 0.56781083, 0.5708365 ,
         0.39193505, 0.44146112, 0.53103083, 0.4784134 ]], dtype=float32),
 array([[0.36720216, 0.32243153, 0.32532868, 0.6119414 , 0.63349384,
         0.33136648, 0.33257827, 0.7090817 , 0.39733034]], dtype=float32),
 array([[0.4704127 , 0.41948444, 0.4416697 , 0.57899725, 0.5480407 ,
         0.39777616, 0.4685875 , 0.5036056 , 0.47486317]], dtype=float32),
 array([[0.36019906, 0.30787262, 0.33253348, 0.6269697 , 0.64577883,
         0.32404166, 0.32958692, 0.7309026 , 0.37360904]], dtype=fl